# For filter 

remove dns from polishedns

In [22]:
lol import os
import logging
import scapy.all as scapy
import random
from collections import defaultdict

os.chdir('LLM4Traffic/tool/Filter')

import logging

logging.basicConfig(       
    level=logging.INFO,            
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',  
    handlers=[
        logging.FileHandler('logs/unpolish_filter_tls.log', mode='w'),  
        logging.StreamHandler()          
    ],
    force=True
)

logger = logging.getLogger()

random.seed(43)

In [23]:
dataset = 'vpn-app'

data_path = f'LLM4Traffic/pipeline/polishedns/{dataset}'

without_dns_output_path = f'LLM4Traffic/pipeline/polished/{dataset}'
un_ratio_output_path = f'LLM4Traffic/pipeline/polishednsLenUn811/{dataset}'
ratio_output_path = f'LLM4Traffic/pipeline/polishednsLen811/{dataset}/train_val_split_0'

os.makedirs(without_dns_output_path, exist_ok=True)
os.makedirs(ratio_output_path, exist_ok=True)

In [25]:
import subprocess

# statistics pcap infomation
def statistics_pcap(pcap_path):
    command = f"tshark -r {pcap_path} -T fields -e _ws.col.Protocol | sort | uniq -c | sort -nr"
    result = subprocess.run(command, shell=True, capture_output=True, text=True)

    # Process the output
    lines = result.stdout.strip().split('\n')
    protocol_counts = {}

    for line in lines:
        count, protocol = line.strip().split(maxsplit=1)
        protocol_counts[protocol] = int(count)

    return protocol_counts

def filter_pcap_without_dns(pcap_path, output_path):
    rule = 'not dns'
    command = f'tshark -r {pcap_path} -Y "{rule}" -w {output_path}'
    subprocess.run(command, shell=True, capture_output=True, text=True)

def filter_pcap_min_length(pcap_path, output_path):
    # because it already remove the ethernet header, so the minimum length is 80 -14 = 66
    rule = 'frame.len >= 66'
    command = f'tshark -r {pcap_path} -Y "{rule}" -w {output_path}'
    subprocess.run(command, shell=True, capture_output=True, text=True)


In [ ]:
for folder in os.listdir(data_path):
    os.makedirs(f'{without_dns_output_path}/{folder}', exist_ok=True)
    
    if folder == 'test':
        for file_name in os.listdir(f'{data_path}/{folder}'):
            logger.info(f'Processing{folder}/{file_name}')
            filter_pcap_without_dns(f'{data_path}/{folder}/{file_name}', f'{without_dns_output_path}/{folder}/{file_name}')
    else:
        for type in os.listdir(f"{data_path}/{folder}"):
            os.makedirs(f'{without_dns_output_path}/{folder}/{type}', exist_ok=True)

            for file_name in os.listdir(f'{data_path}/{folder}/{type}'):
                logger.info(f'Processing {folder}/{type}/{file_name}')
                filter_pcap_without_dns(f'{data_path}/{folder}/{type}/{file_name}', f'{without_dns_output_path}/{folder}/{type}/{file_name}')

In [ ]:
# Mix the train, val, test and 8:1:1 split
from collections import defaultdict

def write2pcap(pkts, output_path):
    with scapy.PcapWriter(output_path, sync=False, append=True) as writer:
        for pkt in pkts:
            writer.write(pkt)

class_list = defaultdict(list)

for folder in os.listdir(without_dns_output_path):
    if folder == 'test':
        for file_name in os.listdir(f'{without_dns_output_path}/{folder}'):
            logger.info(f'Get PATH {folder}/{file_name}')
            class_list[file_name].append(f"{without_dns_output_path}/{folder}/{file_name}")
    else:
        for type in os.listdir(f"{without_dns_output_path}/{folder}"):
            for file_name in os.listdir(f'{without_dns_output_path}/{folder}/{type}'):
                logger.info(f'Get PATH {folder}/{type}/{file_name}')
                class_list[file_name].append(f"{without_dns_output_path}/{folder}/{type}/{file_name}")

print(len(class_list))            

type = ['train', 'val', 'test']

for file_name, paths in class_list.items():
    logger.info(f'Processing {file_name}')
    pkts_list = []
    for path in paths:
        with scapy.PcapReader(path) as pkts:
            for id, pkt in enumerate(pkts):
                if not pkt.haslayer(scapy.Ether):
                    if pkt.haslayer(scapy.IP):
                        eth_header = scapy.Ether(src="00:11:22:33:44:55", dst="66:77:88:99:aa:bb", type=0x0800)
                        pkt = eth_header / pkt
                    elif pkt.haslayer('IPv6'):
                        # print(pkt)
                        eth_header = scapy.Ether(src="00:11:22:33:44:77", dst="66:77:88:99:aa:55", type=0x86DD)
                        pkt = eth_header / pkt
                        # print(pkt)
                    
                # print(id, pkt)
                pkts_list.append(pkt)
        
    num_of_pkts = len(pkts_list)
    random.shuffle(pkts_list)
    train_list = pkts_list[:int(num_of_pkts * 0.8)]
    val_list = pkts_list[int(num_of_pkts * 0.8):int(num_of_pkts * 0.9)]
    test_list = pkts_list[int(num_of_pkts * 0.9):]

    for i, pkts in enumerate([train_list, val_list, test_list]):
        os.makedirs(f'{ratio_output_path}/{type[i]}', exist_ok=True)
        write2pcap(pkts, f"{ratio_output_path}/{type[i]}/{file_name}")

    print(f'{file_name} len of train: {len(train_list)}, len of val: {len(val_list)}, len of test: {len(test_list)}')